In [1]:
#Importing Data from the CSV file
%matplotlib inline
import string
import math
import numpy as np
import matplotlib.pyplot as plt
import pickle

f_train = open("training.1600000.processed.noemoticon.csv", "r", encoding="ISO-8859-1")
X_train = f_train.readlines()
f_test = open("testdata.manual.2009.06.14.csv", "r")
X_test = f_test.readlines()

#Creating the List with just the tweets and finding the number of positive and negative classes. (TRAINING) 
#class_0 = number of classes with label = 0
#class_4 = number of classes with label = 4

class_0=class_4=0
tweets_train =[]
for x in X_train:
    a = x.split('","')
    if (a[0] == '"0'): class_0+=1
    else: class_4+=1
    tweets_train.append('%s"|"%s' % (a[0],a[-1]))

#Creating the List with just the tweets (TEST DATA)
tweets_test =[]
for x in X_test:
    a = x.split('","')
    tweets_test.append('%s"|"%s' % (a[0], a[-1]))
    
print("The length of the training set is = ", len(X_train))
print("The number of classes (label=4) =", class_4)
print("The number of classes (label=0) =", class_0)
print("The length of the test set is = ", len(X_test))

f_test.close()
f_train.close()

m= len(X_train)
phi_0 = class_0/m
phi_4 = class_4/m

The length of the training set is =  1600000
The number of classes (label=4) = 800000
The number of classes (label=0) = 800000
The length of the test set is =  359


In [2]:
import pandas as pd
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectPercentile, chi2
from joblib import parallel_backend


In [3]:
#tweets_train_pd = pd.read_pickle('tweet_train_lemma.pkl')
#tweets_test_pd = pd.read_pickle('tweet_test_lemma.pkl')
train_actual_classes = pd.read_pickle('train_class.pkl')
test_actual_classes = pd.read_pickle('test_class.pkl')

In [4]:
tweets_train_pd = pd.Series(tweets_train)
tweets_train_pd = tweets_train_pd.apply(lambda x: x.split("|")[1])
tweets_test_pd = pd.Series(tweets_test)
tweets_test_pd = tweets_test_pd.apply(lambda x: x.split("|")[1])

In [5]:
train_classes = train_actual_classes.apply(lambda x: 0 if x=='"0"' else 1)
test_classes = test_actual_classes.apply(lambda x: 0 if x=='"0"' else 1)
all_classes = train_classes.unique()

In [12]:
vec = TfidfVectorizer(analyzer="word", stop_words='english',sublinear_tf=True)
tfidf_train = vec.fit_transform(tweets_train_pd)

In [13]:
nb = MultinomialNB(class_prior=[phi_0,phi_4])
nb.fit(tfidf_train, list(train_classes))

MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5], fit_prior=True)

In [14]:
tfidf_test = vec.transform(tweets_test_pd)
test_class_pred = nb.predict(tfidf_test)

In [15]:
def test_accu(test_actual_classes, test_class_pred):
    test_acc=0
    for i in range(len(test_actual_classes)):
        if (list(test_class_pred)[i]==list(test_actual_classes)[i]):
            test_acc+=1
    return(test_acc/len(test_actual_classes))*100


In [16]:
print("The Test Accuracy of the MultinomialNB model trained with TF-IDF is = ", test_accu(test_classes, test_class_pred))

The Test Accuracy of the MultinomialNB model trained with TF-IDF is =  78.55153203342618


In [161]:
vec = TfidfVectorizer(analyzer="word", stop_words='english',sublinear_tf=True, ngram_range=(1,2))
tfidf_train = vec.fit_transform(tweets_train_pd)ravind

In [37]:
nb = MultinomialNB(class_prior=[phi_0,phi_4])
nb.fit(tfidf_train, list(train_classes))

MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5], fit_prior=True)

In [55]:
tfidf_test = vec.transform(tweets_test_pd)
test_class_pred = nb.predict(tfidf_test)

In [179]:
print("The Test Accuracy of the MultinomialNB model trained with TF-IDF and bigrams is= ", nb.score(tfidf_test, test_classes)*100)

The Test Accuracy of the MultinomialNB model trained with TF-IDF and bigrams is=  82.72980501392759


In [180]:
vec = TfidfVectorizer(analyzer="word", stop_words='english',sublinear_tf=True, ngram_range=(1,3))
tfidf_train = vec.fit_transform(tweets_train_pd)

In [181]:
nb = MultinomialNB(class_prior=[phi_0,phi_4])
nb.fit(tfidf_train, list(train_classes))

MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5], fit_prior=True)

In [182]:
tfidf_test = vec.transform(tweets_test_pd)
test_class_pred = nb.predict(tfidf_test)

In [183]:
print("The Test Accuracy of the MultinomialNB model trained with TF-IDF with trigrams is = ", nb.score(tfidf_test, test_classes)*100)

The Test Accuracy of the MultinomialNB model trained with TF-IDF with trigrams is =  82.45125348189416


In [8]:
vec = TfidfVectorizer(analyzer="word", stop_words='english',sublinear_tf=True, min_df=0.001)

tfidf_train = vec.fit_transform(tweets_train_pd)


In [18]:
tfidf_train

<1600000x937 sparse matrix of type '<class 'numpy.float64'>'
	with 6503097 stored elements in Compressed Sparse Row format>

In [15]:
nb = MultinomialNB(class_prior=[phi_0,phi_4])
nb.fit(tfidf_train, list(train_classes))

MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5], fit_prior=True)

In [16]:
tfidf_test = vec.transform(tweets_test_pd)
test_class_pred = nb.predict(tfidf_test)

In [17]:
print("The Test Accuracy of the MultinomialNB model trained with TF-IDF with min_df is = ", nb.score(tfidf_test, test_classes)*100)

The Test Accuracy of the MultinomialNB model trained with TF-IDF with min_df is =  77.99442896935933


In [6]:
def create_minibatch(tweets, classes, batch_size):
    l = len(tweets)
    for i in range(0, l, batch_size):
        X_batch=tweets[i:min(i + batch_size, l)]
        Y_batch=classes[i:min(i + batch_size, l)]
        yield X_batch,Y_batch

In [7]:
model_gauss = GaussianNB()

In [ ]:
#Create minibatches
vec = HashingVectorizer(analyzer="word", stop_words='english',norm='l2')
X = vec.transform(tweets_train_pd)

for (X_batch, Y_batch) in create_minibatch(tweets_train_pd, train_classes, 1000):
    tfidf_train = vec.transform(X_batch)
    model_gauss.partial_fit(tfidf_train.toarray(), Y_batch, classes=all_classes)

In [157]:
X_test = vec.transform(tweets_test_pd)
test_accuracy = model_gauss.score(X_test.toarray(), test_classes)

In [162]:
print("The Test Accuracy of the GaussianNB model trained with Hashing Vectorizer and batch size=1000 is = {}%".format(test_accuracy*100))

The Test Accuracy of the GaussianNB model trained with Hashing Vectorizer and batch size=1000 is = 55.98885793871866%


In [ ]:
vec = HashingVectorizer(analyzer="word", stop_words='english',norm='l2')
X = vec.transform(tweets_train_pd)
print(X.shape)
for (X_batch, Y_batch) in create_minibatch(tweets_train_pd, train_classes, 1000):
    tfidf_train = vec.transform(X_batch)
    print(tfidf_train.shape)
    model_gauss.partial_fit(tfidf_train.toarray(), Y_batch, classes=all_classes)

In [ ]:
#Create minibatches
import time
start = time.time()
vec = HashingVectorizer(analyzer="word", stop_words='english',norm='l2', n_features=2**18)
#X = vec.transform(tweets_train_pd)

for (X_batch, Y_batch) in create_minibatch(tweets_train_pd, train_classes, 1000):
    tfidf_train = vec.transform(X_batch)
    #print(tfidf_train.shape)
    model_gauss.partial_fit(tfidf_train.toarray(), Y_batch, classes=all_classes)

print("The time taken for training on batch size=10000 using hashing vect only=", time.time() - start)

In [ ]:
X_test = vec.transform(tweets_test_pd)
test_accuracy = model_gauss.score(X_test.toarray(), test_classes)

/home/ayushi/.local/lib/python3.6/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: invalid value encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))


In [ ]:
print("The Test Accuracy of the GaussianNB model trained with Hashing Vectorizer and batch size=10000 is = {}%".format(test_accuracy*100))

In [9]:
#Create minibatches

count_vec = CountVectorizer(analyzer="word", stop_words='english')
tfidf_vec = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=True)
#X_count = tfidf_vec.fit_transform(count_vec.fit_transform(tweets_train_pd))
tfidf_transform = (tfidf_vec.fit(count_vec.fit_transform(tweets_train_pd)))

start=time.time()
for (X_batch, Y_batch) in create_minibatch(tweets_train_pd, train_classes, 1000):

    count_batch=count_vec.transform(X_batch)
    tfidf_train_g=tfidf_transform.transform(count_batch)
    model_gauss.partial_fit(tfidf_train_g.toarray(), Y_batch, classes=all_classes)

print("The time taken for training on batch size=100 using tfidf=", time.time() - start)    

The time taken for training on batch size=100 using tfidf= 8480.386967897415


In [19]:
X_test = tfidf_transform.transform(count_vec.transform(tweets_test_pd))
test_accuracy = model_gauss.score(X_test.toarray(), test_classes)

In [20]:
print("The Test Accuracy of the GaussianNB model trained with TFIDF and batch size=100 is = {}%".format(test_accuracy*100))

The Test Accuracy of the GaussianNB model trained with TFIDF and batch size=100 is = 51.25348189415042%


In [10]:
print("The Test Accuracy of the GaussianNB model trained with TFIDF and batch size=1000 is = {}%".format(test_accuracy*100))

The Test Accuracy of the GaussianNB model trained with TFIDF and batch size=1000 is = 51.25348189415042%


In [8]:
tweets_shuffled = pd.concat([tweets_train_pd, train_classes], axis=1).sample(frac=1)

In [9]:
tweets_train = list(tweets_shuffled[0])
train_classes = list(tweets_shuffled[1])

In [15]:
tfidf_vec = TfidfVectorizer(analyzer="word", stop_words='english',sublinear_tf=True, min_df=0.001)
tfidf_train = tfidf_vec.fit_transform(tweets_train)

In [ ]:
model_gauss.fit(tfidf_train.toarray(), train_classes)

In [14]:
X_test = tfidf_vec.transform(tweets_test_pd)
test_accuracy = model_gauss.score(X_test.toarray(), test_classes)
print(test_accuracy) #min-df=0.001

0.7771587743732591


In [10]:
count_vec = CountVectorizer(analyzer="word", stop_words='english', min_df=0.001)
tfidf_vec = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=True)
X_count = tfidf_vec.fit_transform(count_vec.fit_transform(tweets_train))
#tfidf_transform = (tfidf_vec.fit(count_vec.fit_transform(tweets_train)))
model_gauss.fit(X_count.toarray(), train_classes)
#Also gives 77.71% accuracy

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
#Create minibatches

count_vec = CountVectorizer(analyzer="word", stop_words='english', min_df=0.00001)
tfidf_vec = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=True)
#X_count = tfidf_vec.fit_transform(count_vec.fit_transform(tweets_train_pd))
tfidf_transform = (tfidf_vec.fit(count_vec.fit_transform(tweets_train)))

start=time.time()
for (X_batch, Y_batch) in create_minibatch(tweets_train, train_classes, 1000):

    count_batch=count_vec.transform(X_batch)
    tfidf_train_g=tfidf_transform.transform(count_batch)
    model_gauss.partial_fit(tfidf_train_g.toarray(), Y_batch, classes=all_classes)

print("The time taken for training on batch size=10000 using tfidf=", time.time() - start)    

In [11]:
X_test = tfidf_transform.transform(count_vec.transform(tweets_test_pd))
test_accuracy = model_gauss.score(X_test.toarray(), test_classes)

In [12]:
print("The Test Accuracy of the GaussianNB model trained with TFIDF and batch size=10000 is = {}%".format(test_accuracy*100))

The Test Accuracy of the GaussianNB model trained with TFIDF and batch size=10000 is = 74.65181058495823%


In [ ]:
vec = TfidfVectorizer(analyzer="word", stop_words='english',sublinear_tf=True, ngram_range=(1,1))
tfidf_train = vec.fit_transform(tweets_train_pd)
tfidf_train_new = SelectPercentile(chi2, percentile=50).fit_transform(tfidf_train, train_classes)
tfidf_train_new

In [11]:

with parallel_backend('threading', n_jobs=2):
    # Your scikit-learn code here

SyntaxError: unexpected EOF while parsing (<ipython-input-11-e3be00bc68e6>, line 4)